In [1]:
from pyspark.sql import SparkSession

In [3]:
sc = SparkSession.builder.getOrCreate()

In [4]:
sdf = sc.read.json('nyt2.json')

In [9]:
sdf.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|Little, Brown| {2}|           {1}|           

In [10]:
sdf_dropup = sdf.dropDuplicates()
sdf_dropup.show(10)

+--------------------+--------------------+------------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+-------------------+-------------+
|                 _id|  amazon_product_url|            author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|              title|weeks_on_list|
+--------------------+--------------------+------------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+-------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|      Daniel Silva|{{1217030400000}}|Gabriel Allon, an...|{26.95, null}|{{1218326400000}}|       Putnam| {1}|           {0}| THE SECRET SERVANT|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|        Jane Green|{{1218240000000}}|A woman’s life ch...|    {null, 0}|{{1219536000000}}|       Viking|{18}|           {0}|    THE BEACH HOUSE|     

In [11]:
sdf.select("author").show(10)

+--------------------+
|              author|
+--------------------+
|       Dean R Koontz|
|     Stephenie Meyer|
|        Emily Giffin|
|   Patricia Cornwell|
|     Chuck Palahniuk|
|James Patterson a...|
|       John Sandford|
|       Jimmy Buffett|
|    Elizabeth George|
|      David Baldacci|
+--------------------+
only showing top 10 rows



In [12]:
sdf.select("author","title","rank","price").show(10)

+--------------------+--------------------+----+-------------+
|              author|               title|rank|        price|
+--------------------+--------------------+----+-------------+
|       Dean R Koontz|           ODD HOURS| {1}|   {null, 27}|
|     Stephenie Meyer|            THE HOST| {2}|{25.99, null}|
|        Emily Giffin|LOVE THE ONE YOU'...| {3}|{24.95, null}|
|   Patricia Cornwell|           THE FRONT| {4}|{22.95, null}|
|     Chuck Palahniuk|               SNUFF| {5}|{24.95, null}|
|James Patterson a...|SUNDAYS AT TIFFANY’S| {6}|{24.99, null}|
|       John Sandford|        PHANTOM PREY| {7}|{26.95, null}|
|       Jimmy Buffett|          SWINE NOT?| {8}|{21.99, null}|
|    Elizabeth George|     CARELESS IN RED| {9}|{27.95, null}|
|      David Baldacci|     THE WHOLE TRUTH|{10}|{26.99, null}|
+--------------------+--------------------+----+-------------+
only showing top 10 rows



In [16]:
from pyspark.sql.functions import when

sdf.select("title", when(sdf.title != "ODD HOURS",1).otherwise(0)).show(10)

+--------------------+-----------------------------------------------------+
|               title|CASE WHEN (NOT (title = ODD HOURS)) THEN 1 ELSE 0 END|
+--------------------+-----------------------------------------------------+
|           ODD HOURS|                                                    0|
|            THE HOST|                                                    1|
|LOVE THE ONE YOU'...|                                                    1|
|           THE FRONT|                                                    1|
|               SNUFF|                                                    1|
|SUNDAYS AT TIFFANY’S|                                                    1|
|        PHANTOM PREY|                                                    1|
|          SWINE NOT?|                                                    1|
|     CARELESS IN RED|                                                    1|
|     THE WHOLE TRUTH|                                                    1|

In [19]:
sdf[sdf.author.isin("John Sandford","Emily Giffin")].select("author").show(10)

+-------------+
|       author|
+-------------+
| Emily Giffin|
|John Sandford|
| Emily Giffin|
|John Sandford|
| Emily Giffin|
|John Sandford|
| Emily Giffin|
| Emily Giffin|
|John Sandford|
| Emily Giffin|
+-------------+
only showing top 10 rows



In [22]:
sdf.select("author","title",sdf.title.like("THE %")).show(15)

+--------------------+--------------------+----------------+
|              author|               title|title LIKE THE %|
+--------------------+--------------------+----------------+
|       Dean R Koontz|           ODD HOURS|           false|
|     Stephenie Meyer|            THE HOST|            true|
|        Emily Giffin|LOVE THE ONE YOU'...|           false|
|   Patricia Cornwell|           THE FRONT|            true|
|     Chuck Palahniuk|               SNUFF|           false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|           false|
|       John Sandford|        PHANTOM PREY|           false|
|       Jimmy Buffett|          SWINE NOT?|           false|
|    Elizabeth George|     CARELESS IN RED|           false|
|      David Baldacci|     THE WHOLE TRUTH|            true|
|        Troy Denning|          INVINCIBLE|           false|
|          James Frey|BRIGHT SHINY MORNING|           false|
|         Garth Stein|THE ART OF RACING...|            true|
|     Debbie Macomber|  

In [23]:
sdf.select("author", "title", sdf.title.startswith("THE")).show(5)

+-----------------+--------------------+----------------------+
|           author|               title|startswith(title, THE)|
+-----------------+--------------------+----------------------+
|    Dean R Koontz|           ODD HOURS|                 false|
|  Stephenie Meyer|            THE HOST|                  true|
|     Emily Giffin|LOVE THE ONE YOU'...|                 false|
|Patricia Cornwell|           THE FRONT|                  true|
|  Chuck Palahniuk|               SNUFF|                 false|
+-----------------+--------------------+----------------------+
only showing top 5 rows



In [24]:
sdf.select("author", "title", sdf.title.endswith("NT")).show(5)

+-----------------+--------------------+-------------------+
|           author|               title|endswith(title, NT)|
+-----------------+--------------------+-------------------+
|    Dean R Koontz|           ODD HOURS|              false|
|  Stephenie Meyer|            THE HOST|              false|
|     Emily Giffin|LOVE THE ONE YOU'...|              false|
|Patricia Cornwell|           THE FRONT|               true|
|  Chuck Palahniuk|               SNUFF|              false|
+-----------------+--------------------+-------------------+
only showing top 5 rows



In [26]:
sdf.select(sdf.author.substr(1,3).alias("author")).show(5)

+------+
|author|
+------+
|   Dea|
|   Ste|
|   Emi|
|   Pat|
|   Chu|
+------+
only showing top 5 rows



In [27]:
sdf.select(sdf.author.substr(3,6).alias("author")).show(5)

+------+
|author|
+------+
|an R K|
|epheni|
|ily Gi|
|tricia|
|uck Pa|
+------+
only showing top 5 rows



In [28]:
sdf.select(sdf.author.substr(1,6).alias("author")).show(5)

+------+
|author|
+------+
|Dean R|
|Stephe|
|Emily |
|Patric|
|Chuck |
+------+
only showing top 5 rows



In [29]:
from pyspark.sql.functions import lit

sdf = sdf.withColumn('new_column', lit('This is a new column'))
display(sdf)

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

In [30]:
sdf.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, 

In [31]:
sdf = sdf.withColumnRenamed('amazon_product_url','URL')
sdf.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, 

In [32]:
sdf_remove = sdf.drop("publisher","published_date").show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|        price|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}| {1}|           {0}|           ODD HOURS|          {1}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}| {2}|           {1}|            THE HOST|          {3}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  

In [34]:
sdf_remove2 = sdf.drop(sdf.publisher).drop(sdf.published_date).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|        price|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}| {1}|           {0}|           ODD HOURS|          {1}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}| {2}|           {1}|            THE HOST|          {3}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  

In [36]:
sdf.dtypes

[('_id', 'struct<$oid:string>'),
 ('URL', 'string'),
 ('author', 'string'),
 ('bestsellers_date', 'struct<$date:struct<$numberLong:string>>'),
 ('description', 'string'),
 ('price', 'struct<$numberDouble:string,$numberInt:string>'),
 ('published_date', 'struct<$date:struct<$numberLong:string>>'),
 ('publisher', 'string'),
 ('rank', 'struct<$numberInt:string>'),
 ('rank_last_week', 'struct<$numberInt:string>'),
 ('title', 'string'),
 ('weeks_on_list', 'struct<$numberInt:string>'),
 ('new_column', 'string')]

In [37]:
sdf.describe().show()

+-------+--------------------+---------------+--------------------+---------+------------------+--------------------+
|summary|                 URL|         author|         description|publisher|             title|          new_column|
+-------+--------------------+---------------+--------------------+---------+------------------+--------------------+
|  count|               10195|          10195|               10195|    10195|             10195|               10195|
|   mean|                null|           null|                null|     null|1877.7142857142858|                null|
| stddev|                null|           null|                null|     null| 370.9760613506458|                null|
|    min|http://www.amazon...|        AJ Finn|                    |      ACE|  10TH ANNIVERSARY|This is a new column|
|    max|https://www.amazo...|various authors|’Tis for the Rebe...|allantine|               ZOO|This is a new column|
+-------+--------------------+---------------+----------

In [38]:
sdf.columns

['_id',
 'URL',
 'author',
 'bestsellers_date',
 'description',
 'price',
 'published_date',
 'publisher',
 'rank',
 'rank_last_week',
 'title',
 'weeks_on_list',
 'new_column']

In [39]:
sdf.count()

10195

In [40]:
sdf.distinct().count()

10195

In [42]:
sdf.explain()

== Physical Plan ==
*(1) Project [_id#7, amazon_product_url#8 AS URL#624, author#9, bestsellers_date#10, description#11, price#12, published_date#13, publisher#14, rank#15, rank_last_week#16, title#17, weeks_on_list#18, This is a new column AS new_column#545]
+- FileScan json [_id#7,amazon_product_url#8,author#9,bestsellers_date#10,description#11,price#12,published_date#13,publisher#14,rank#15,rank_last_week#16,title#17,weeks_on_list#18] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/david/python/spark/nyt2.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_id:struct<$oid:string>,amazon_product_url:string,author:string,bestsellers_date:struct<$d...




In [43]:
sdf.groupBy("author").count().show(10)

+-----------------+-----+
|           author|count|
+-----------------+-----+
|       James Frey|    2|
| Elin Hilderbrand|   58|
|Sharon Kay Penman|    2|
|      Lisa Genova|    7|
|     Will Allison|    1|
|Patricia Cornwell|   64|
|    Laurie R King|    6|
|       Tea Obreht|    6|
|     Sarah Dunant|    1|
|     Tim Johnston|    1|
+-----------------+-----+
only showing top 10 rows



In [45]:
sdf.filter(sdf.title == 'THE HOST').show(5)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|   title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|Little, Brown| {2}|           {1}|THE HOST|          {3}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1212192000000}}|Aliens have taken...|{25.99, null}|{{1213488000000}}|Little, Brown| {2}|           {2}|

In [47]:
sdf.na.fill(0)
sdf.fillna(0)

DataFrame[_id: struct<$oid:string>, URL: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

In [48]:
sdf.toPandas()

,_id,URL,author,bestsellers_date,description,price,published_date,publisher,rank,rank_last_week,title,weeks_on_list,new_column
0,"(5b4aa4ead3089013507db18b,)",http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"((1211587200000,),)","Odd Thomas, who can communicate with the dead,...","(None, 27)","((1212883200000,),)",Bantam,"(1,)","(0,)",ODD HOURS,"(1,)",This is a new column
1,"(5b4aa4ead3089013507db18c,)",http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,"((1211587200000,),)",Aliens have taken control of the minds and bod...,"(25.99, None)","((1212883200000,),)","Little, Brown","(2,)","(1,)",THE HOST,"(3,)",This is a new column
2,"(5b4aa4ead3089013507db18d,)",http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,"((1211587200000,),)",A woman's happy marriage is shaken when she en...,"(24.95, None)","((1212883200000,),)",St. Martin's,"(3,)","(2,)",LOVE THE ONE YOU'RE WITH,"(2,)",This is a new column
3,"(5b4aa4ead3089013507db18e,)",http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,"((1211587200000,),)",A Massachusetts state investigator and his tea...,"(22.95, None)","((1212883200000,),)",Putnam,"(4,)","(0,)",THE FRONT,"(1,)",This is a new column
4,"(5b4aa4ead3089013507db18f,)",http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,"((1211587200000,),)",An aging porn queens aims to cap her career by...,"(24.95, None)","((1212883200000,),)",Doubleday,"(5,)","(0,)",SNUFF,"(1,)",This is a new column
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10190,"(5b4aa4ead3089013507dd959,)",https://www.amazon.com/Clancy-Line-Sight-Jack-...,Mike Maden,"((1530921600000,),)",Jack Ryan Jr. risks his life to protect a woma...,"(None, 0)","((1532217600000,),)",Putnam,"(11,)","(6,)",TOM CLANCY LINE OF SIGHT,"(4,)",This is a new column
10191,"(5b4aa4ead3089013507dd95a,)",https://www.amazon.com/Something-Water-Novel-C...,Catherine Steadman,"((1530921600000,),)",A documentary filmmaker and an investment bank...,"(None, 0)","((1532217600000,),)",Ballantine,"(12,)","(11,)",SOMETHING IN THE WATER,"(5,)",This is a new column
10192,"(5b4aa4ead3089013507dd95b,)",https://www.amazon.com/Little-Fires-Everywhere...,Celeste Ng,"((1530921600000,),)",An artist upends a quiet town outside Cleveland.,"(None, 0)","((1532217600000,),)",Penguin Press,"(13,)","(12,)",LITTLE FIRES EVERYWHERE,"(41,)",This is a new column
10193,"(5b4aa4ead3089013507dd95c,)",https://www.amazon.com/Shelter-Place-Nora-Robe...,Nora Roberts,"((1530921600000,),)",Survivors of a mass shooting outside a mall in...,"(None, 0)","((1532217600000,),)",St. Martin's,"(14,)","(5,)",SHELTER IN PLACE,"(6,)",This is a new column


In [49]:
sc.stop()